In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import transformers as ppb
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

### Importing the dataset

In [ ]:
# read train.EN.csv into a dataframe
df = pd.read_csv('https://raw.githubusercontent.com/iabufarha/iSarcasmEval/main/train/train.En.csv', index_col=False)

# read test.EN.csv into a dataframe
df_test = pd.read_csv('https://raw.githubusercontent.com/iabufarha/iSarcasmEval/main/test/task_A_En_test.csv', index_col=False)

# Shuffle data
df = df.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

In [ ]:
df1 = df[['tweet', 'sarcastic']]
df2 = df_test[['text', 'sarcastic']]

In [ ]:
df1.head(1)

,tweet,sarcastic
0,y’all were all vote-blue-no-matter-who till be...,0


In [ ]:
batch_1 = df1

### Loading the Pre-trained TwHinBERT model

In [ ]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("Twitter/TwHIN-BERT-base")
threshold = 0.475

# For transformers v4.x+:
tokenizer = AutoTokenizer.from_pretrained("Twitter/TwHIN-BERT-base")

Some weights of the model checkpoint at Twitter/TwHIN-BERT-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at Twitter/TwHIN-BERT-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weig

## Model #1: TwHinBERT

We first use TwHinBERT to find the [CLS] tokens for all input sentences

### Tokenization

In [ ]:
tokenized = batch_1['tweet'].apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))
tokenized_test = df2['text'].apply((lambda x: tokenizer.encode(str(x), add_special_tokens=True)))

### Padding

In [ ]:
# Pad to the same size for train data
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

# Pad to the same size for test data
max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)
      
padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [ ]:
print(np.array(padded).shape, np.array(padded_test).shape)

(3468, 121) (1400, 157)


### Masking

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_test_mask = np.where(padded_test != 0, 1, 0)
print(attention_mask.shape, attention_test_mask.shape)

(3468, 121) (1400, 157)


### Embedding

In [ ]:
# Use TwHinBERT to embed all training data
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
# Use TwHinBERT to embed all test data
input_ids_test = torch.tensor(padded_test)
attention_test_mask = torch.tensor(attention_test_mask)

with torch.no_grad():
    last_hidden_states_test = model(input_ids_test, attention_mask=attention_test_mask)

In [ ]:
# Save [CLS] tokens for classification
# last_hidden_states[0][sentences, token position [CLS] as the first one, 768 units]
train_features = last_hidden_states[0][:,0,:].numpy()
test_features = last_hidden_states_test[0][:,0,:].numpy()

In [ ]:
train_labels = batch_1['sarcastic'].to_numpy()
test_labels = df2['sarcastic'].to_numpy()

## Model #2: MLP

In [ ]:
class MainNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.layernorm1 = nn.LayerNorm(768)
        self.fc1 = nn.Linear(768, 64)
        self.layernorm2 = nn.LayerNorm(64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_):
        out = self.layernorm1(input_)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        
        return out

In [ ]:
def train_epoch(model, opt, criterion, batch_size=len(train_features)):
    model.train()
    losses = []
    for beg_i in range(0, len(train_features), batch_size):
        x_batch = torch.from_numpy(train_features[beg_i:beg_i + batch_size, :])
        y_true = train_labels[beg_i:beg_i + batch_size]
        y_batch = torch.from_numpy(train_labels[beg_i:beg_i + batch_size])
        x_batch = Variable(x_batch)
        y_batch = Variable(y_batch)
        y_batch = y_batch.unsqueeze(1)
        y_batch = y_batch.float()

        opt.zero_grad()
        y_hat = model(x_batch)
        loss = criterion(y_hat, y_batch)
        loss.backward()
        opt.step()
        losses.append(loss.data.numpy())

    return losses

In [ ]:
test_features, valid_features, test_labels, valid_labels = train_test_split(test_features, test_labels, test_size = 0.1, random_state=256)

In [ ]:
e_losses = []
num_epochs = 5
net = MainNet()
opt = optim.Adam(net.parameters(), lr=0.00001, betas=(0.9, 0.999))
criterion = nn.BCELoss()
for e in range(num_epochs):
    e_losses += train_epoch(net, opt, criterion)
#plt.plot(e_losses)

## Evaluating Model

In [ ]:
# Evaluate the test set and print the accuracy and f1 score
test_features_t = torch.from_numpy(test_features)
test_labels_t = torch.from_numpy(test_labels)
test_labels_t = test_labels_t.unsqueeze(1)
test_labels_t = test_labels_t.float()
net.eval()
y_hat = net(test_features_t)
y_hat = y_hat.detach().numpy()
y_hat = np.where(y_hat <= threshold, 0, 1)
y_hat = np.round(y_hat)
print(f1_score(test_labels, y_hat))

F1 Score:  0.0
0.2826666666666667 0.47500000000000003


In [ ]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

y_hat = lr_clf.predict(valid_features)
print("F1 Score: ", f1_score(valid_labels, y_hat))

F1 Score:  0.17777777777777778
